In [1]:
import os
import pandas as pd
import numpy as np
from tabulate import tabulate
from datetime import datetime
from sklearn.impute import SimpleImputer
from tabula import read_pdf
import math

In [113]:
df = pd.read_csv('finalcsv.csv', index_col=0)

In [114]:
df

,state,county,population,buildvalue,agrivalue,area,risk_score,risk_rating,risk_npctl,risk_spctl,...,storm_risks,full_state,revenue_per_person,state_funding,state.1,county.1,drgt_ealt,hrcn_ealt,swnd_ealt,ltng_ealt
0,AL,Autauga,54571,5.075584e+09,21460000.0,594.448312,5.748777,Very Low,16.359007,5.970149,...,7.631697,Alabama,3756,204968676,Kentucky,Johnson,1825.417662,3272.974160,1.548040e+05,46448.730787
1,AL,Baldwin,182265,2.314295e+10,120383000.0,1589.815853,13.557905,Relatively Moderate,79.344367,82.089552,...,23.224606,Alabama,3756,684587340,Kentucky,Kenton,4641.483953,9185.266739,1.874503e+06,37762.813970
2,AL,Barbour,27457,2.206426e+09,105565000.0,885.001636,9.419639,Relatively Low,52.959898,41.791045,...,12.471689,Alabama,3756,103128492,Kentucky,Knott,1555.704677,615.508974,1.448994e+05,32220.028068
3,AL,Bibb,22915,1.643641e+09,4199000.0,622.460425,4.346314,Very Low,7.192871,1.492537,...,5.382791,Alabama,3756,86068740,Kentucky,Knox,17459.131710,2594.946558,3.297535e+05,105586.063748
4,AL,Blount,57322,4.794278e+09,242855000.0,644.821022,9.362945,Relatively Low,52.546149,40.298507,...,13.540158,Alabama,3756,215301432,Kentucky,Larue,23440.487786,17810.245704,3.498765e+05,38895.410938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,WY,Sweetwater,43806,4.583896e+09,16454000.0,10426.919825,2.572885,Very Low,1.845958,13.043478,...,3.481503,Wyoming,5353,234493518,Wyoming,Sweetwater,67258.895675,NaN,2.931848e+04,36578.975784
3138,WY,Teton,21294,3.476594e+09,17582000.0,3996.855337,6.513988,Very Low,23.551878,60.869565,...,6.210718,Wyoming,5353,113986782,Wyoming,Teton,32886.299165,NaN,2.227379e+04,188154.050727
3139,WY,Uinta,21118,2.018844e+09,26662000.0,2081.651840,4.458528,Very Low,7.702101,30.434783,...,5.590674,Wyoming,5353,113044654,Wyoming,Uinta,175826.688938,NaN,4.847070e+04,39079.410356
3140,WY,Washakie,8533,8.867450e+08,43019000.0,2238.665800,5.182977,Very Low,11.807766,47.826087,...,7.378384,Wyoming,5353,45677149,Wyoming,Washakie,223781.785275,NaN,1.598940e+04,22528.317388


In [115]:
df.columns

Index(['state', 'county', 'population', 'buildvalue', 'agrivalue', 'area',
       'risk_score', 'risk_rating', 'risk_npctl', 'risk_spctl', 'resl_score',
       'resl_ratng', 'resl_npctl', 'resl_spctl', 'resl_value', 'drgt_evnts',
       'drgt_afreq', 'drgt_risks', 'hrcn_evnts', 'hrcn_afreq', 'hrcn_risks',
       'storm_evnts', 'storm_afreq', 'storm_risks', 'full_state',
       'revenue_per_person', 'state_funding', 'state.1', 'county.1',
       'drgt_ealt', 'hrcn_ealt', 'swnd_ealt', 'ltng_ealt'],
      dtype='object')

In [116]:
df.rename(columns={'drgt_evnts':'drought_events',
                   'drgt_afreq':'drought_freq',
                   'drgt_risks':'drought_score',
                   'hrcn_evnts':'hurricane_events',
                   'hrcn_afreq':'hurricane_freq',
                   'hrcn_risks':'hurricane_score',
                   'storm_evnts':'storm_events',
                   'storm_afreq':'storm_freq',
                   'storm_risks':'storm_score'}, inplace=True)

In [117]:
df.head()

,state,county,population,buildvalue,agrivalue,area,risk_score,risk_rating,risk_npctl,risk_spctl,...,storm_score,full_state,revenue_per_person,state_funding,state.1,county.1,drgt_ealt,hrcn_ealt,swnd_ealt,ltng_ealt
0,AL,Autauga,54571,5.075584e+09,21460000.0,594.448312,5.748777,Very Low,16.359007,5.970149,...,7.631697,Alabama,3756,204968676,Kentucky,Johnson,1825.417662,3272.974160,1.548040e+05,46448.730787
1,AL,Baldwin,182265,2.314295e+10,120383000.0,1589.815853,13.557905,Relatively Moderate,79.344367,82.089552,...,23.224606,Alabama,3756,684587340,Kentucky,Kenton,4641.483953,9185.266739,1.874503e+06,37762.813970
2,AL,Barbour,27457,2.206426e+09,105565000.0,885.001636,9.419639,Relatively Low,52.959898,41.791045,...,12.471689,Alabama,3756,103128492,Kentucky,Knott,1555.704677,615.508974,1.448994e+05,32220.028068
3,AL,Bibb,22915,1.643641e+09,4199000.0,622.460425,4.346314,Very Low,7.192871,1.492537,...,5.382791,Alabama,3756,86068740,Kentucky,Knox,17459.131710,2594.946558,3.297535e+05,105586.063748
4,AL,Blount,57322,4.794278e+09,242855000.0,644.821022,9.362945,Relatively Low,52.546149,40.298507,...,13.540158,Alabama,3756,215301432,Kentucky,Larue,23440.487786,17810.245704,3.498765e+05,38895.410938


In [118]:
df = df.drop(columns={'risk_rating'})
df.head()

,state,county,population,buildvalue,agrivalue,area,risk_score,risk_npctl,risk_spctl,resl_score,...,storm_score,full_state,revenue_per_person,state_funding,state.1,county.1,drgt_ealt,hrcn_ealt,swnd_ealt,ltng_ealt
0,AL,Autauga,54571,5.075584e+09,21460000.0,594.448312,5.748777,16.359007,5.970149,55.529800,...,7.631697,Alabama,3756,204968676,Kentucky,Johnson,1825.417662,3272.974160,1.548040e+05,46448.730787
1,AL,Baldwin,182265,2.314295e+10,120383000.0,1589.815853,13.557905,79.344367,82.089552,54.759202,...,23.224606,Alabama,3756,684587340,Kentucky,Kenton,4641.483953,9185.266739,1.874503e+06,37762.813970
2,AL,Barbour,27457,2.206426e+09,105565000.0,885.001636,9.419639,52.959898,41.791045,50.781998,...,12.471689,Alabama,3756,103128492,Kentucky,Knott,1555.704677,615.508974,1.448994e+05,32220.028068
3,AL,Bibb,22915,1.643641e+09,4199000.0,622.460425,4.346314,7.192871,1.492537,51.807600,...,5.382791,Alabama,3756,86068740,Kentucky,Knox,17459.131710,2594.946558,3.297535e+05,105586.063748
4,AL,Blount,57322,4.794278e+09,242855000.0,644.821022,9.362945,52.546149,40.298507,53.357400,...,13.540158,Alabama,3756,215301432,Kentucky,Larue,23440.487786,17810.245704,3.498765e+05,38895.410938


In [119]:
df2 = df.sort_values(by=['full_state','county']).reset_index(drop=True)
df2

,state,county,population,buildvalue,agrivalue,area,risk_score,risk_npctl,risk_spctl,resl_score,...,storm_score,full_state,revenue_per_person,state_funding,state.1,county.1,drgt_ealt,hrcn_ealt,swnd_ealt,ltng_ealt
0,AL,Autauga,54571,5.075584e+09,21460000.0,594.448312,5.748777,16.359007,5.970149,55.529800,...,7.631697,Alabama,3756,204968676,Kentucky,Johnson,1825.417662,3272.974160,1.548040e+05,46448.730787
1,AL,Baldwin,182265,2.314295e+10,120383000.0,1589.815853,13.557905,79.344367,82.089552,54.759202,...,23.224606,Alabama,3756,684587340,Kentucky,Kenton,4641.483953,9185.266739,1.874503e+06,37762.813970
2,AL,Barbour,27457,2.206426e+09,105565000.0,885.001636,9.419639,52.959898,41.791045,50.781998,...,12.471689,Alabama,3756,103128492,Kentucky,Knott,1555.704677,615.508974,1.448994e+05,32220.028068
3,AL,Bibb,22915,1.643641e+09,4199000.0,622.460425,4.346314,7.192871,1.492537,51.807600,...,5.382791,Alabama,3756,86068740,Kentucky,Knox,17459.131710,2594.946558,3.297535e+05,105586.063748
4,AL,Blount,57322,4.794278e+09,242855000.0,644.821022,9.362945,52.546149,40.298507,53.357400,...,13.540158,Alabama,3756,215301432,Kentucky,Larue,23440.487786,17810.245704,3.498765e+05,38895.410938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,WY,Sweetwater,43806,4.583896e+09,16454000.0,10426.919825,2.572885,1.845958,13.043478,51.255002,...,3.481503,Wyoming,5353,234493518,Wyoming,Sweetwater,67258.895675,NaN,2.931848e+04,36578.975784
3138,WY,Teton,21294,3.476594e+09,17582000.0,3996.855337,6.513988,23.551878,60.869565,51.918402,...,6.210718,Wyoming,5353,113986782,Wyoming,Teton,32886.299165,NaN,2.227379e+04,188154.050727
3139,WY,Uinta,21118,2.018844e+09,26662000.0,2081.651840,4.458528,7.702101,30.434783,52.554202,...,5.590674,Wyoming,5353,113044654,Wyoming,Uinta,175826.688938,NaN,4.847070e+04,39079.410356
3140,WY,Washakie,8533,8.867450e+08,43019000.0,2238.665800,5.182977,11.807766,47.826087,51.484398,...,7.378384,Wyoming,5353,45677149,Wyoming,Washakie,223781.785275,NaN,1.598940e+04,22528.317388


---

In [9]:
f = pd.read_csv('state_funding_2022.csv', index_col=0)
f.head()

,state,funding,%_of_total
0,Alabama (AL),"$54,697,779,694",1.33%
1,Alaska (AK),"$13,617,642,444",0.33%
2,Arizona (AZ),"$97,000,340,211",2.36%
3,Arkansas (AR),"$29,378,338,658",0.72%
4,California (CA),"$390,811,907,053",9.52%


In [10]:
f.iloc[:51,:]

,state,funding,%_of_total
0,Alabama (AL),"$54,697,779,694",1.33%
1,Alaska (AK),"$13,617,642,444",0.33%
2,Arizona (AZ),"$97,000,340,211",2.36%
3,Arkansas (AR),"$29,378,338,658",0.72%
4,California (CA),"$390,811,907,053",9.52%
5,Colorado (CO),"$53,910,424,226",1.31%
6,Connecticut (CT),"$104,360,084,694",2.54%
7,Delaware (DE),"$9,905,940,162",0.24%
8,District of Columbia (DC),"$48,804,556,621",1.19%
9,Florida (FL),"$239,415,273,788",5.83%


In [13]:
f = f.iloc[:51,:]

In [14]:
f.rename(columns={'state':'s'}, inplace=True)

['AL', 'AK','AZ','AR','CA','CO','CT','DE','D.C.','FL','GA',
 'HI','ID','IL','IN','IA','KS','KY','LA','ME','MD',
 'MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ',
 'NM','NY','NC','ND','OH','OK','OR','PA','RI','SC',
 'SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']

In [15]:
f['state'] = ['AL', 'AK','AZ','AR','CA','CO','CT','DE','D.C.','FL','GA',
 'HI','ID','IL','IN','IA','KS','KY','LA','ME','MD',
 'MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ',
 'NM','NY','NC','ND','OH','OK','OR','PA','RI','SC',
 'SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']
f

,s,funding,%_of_total,state
0,Alabama (AL),"$54,697,779,694",1.33%,AL
1,Alaska (AK),"$13,617,642,444",0.33%,AK
2,Arizona (AZ),"$97,000,340,211",2.36%,AZ
3,Arkansas (AR),"$29,378,338,658",0.72%,AR
4,California (CA),"$390,811,907,053",9.52%,CA
5,Colorado (CO),"$53,910,424,226",1.31%,CO
6,Connecticut (CT),"$104,360,084,694",2.54%,CT
7,Delaware (DE),"$9,905,940,162",0.24%,DE
8,District of Columbia (DC),"$48,804,556,621",1.19%,D.C.
9,Florida (FL),"$239,415,273,788",5.83%,FL


In [16]:
f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 0 to 50
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   s           51 non-null     object
 1   funding     51 non-null     object
 2   %_of_total  51 non-null     object
 3   state       51 non-null     object
dtypes: object(4)
memory usage: 2.0+ KB


In [25]:
f.funding = f.funding.str.replace('$','', regex=True).replace(',','', regex=True)

In [29]:
f.funding = f.funding.astype(int)

In [30]:
f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 0 to 50
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   s           51 non-null     object
 1   funding     51 non-null     int64 
 2   %_of_total  51 non-null     object
 3   state       51 non-null     object
dtypes: int64(1), object(3)
memory usage: 2.0+ KB


In [120]:
df3 = df2.merge(f, how='right', on='state')
df3

,state,county,population,buildvalue,agrivalue,area,risk_score,risk_npctl,risk_spctl,resl_score,...,state_funding,state.1,county.1,drgt_ealt,hrcn_ealt,swnd_ealt,ltng_ealt,s,funding,%_of_total
0,AL,Autauga,54571,5.075584e+09,21460000.0,594.448312,5.748777,16.359007,5.970149,55.529800,...,204968676,Kentucky,Johnson,1825.417662,3272.974160,1.548040e+05,46448.730787,Alabama (AL),54697779694,1.33%
1,AL,Baldwin,182265,2.314295e+10,120383000.0,1589.815853,13.557905,79.344367,82.089552,54.759202,...,684587340,Kentucky,Kenton,4641.483953,9185.266739,1.874503e+06,37762.813970,Alabama (AL),54697779694,1.33%
2,AL,Barbour,27457,2.206426e+09,105565000.0,885.001636,9.419639,52.959898,41.791045,50.781998,...,103128492,Kentucky,Knott,1555.704677,615.508974,1.448994e+05,32220.028068,Alabama (AL),54697779694,1.33%
3,AL,Bibb,22915,1.643641e+09,4199000.0,622.460425,4.346314,7.192871,1.492537,51.807600,...,86068740,Kentucky,Knox,17459.131710,2594.946558,3.297535e+05,105586.063748,Alabama (AL),54697779694,1.33%
4,AL,Blount,57322,4.794278e+09,242855000.0,644.821022,9.362945,52.546149,40.298507,53.357400,...,215301432,Kentucky,Larue,23440.487786,17810.245704,3.498765e+05,38895.410938,Alabama (AL),54697779694,1.33%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,WY,Sweetwater,43806,4.583896e+09,16454000.0,10426.919825,2.572885,1.845958,13.043478,51.255002,...,234493518,Wyoming,Sweetwater,67258.895675,NaN,2.931848e+04,36578.975784,Wyoming (WY),6448803714,0.16%
3138,WY,Teton,21294,3.476594e+09,17582000.0,3996.855337,6.513988,23.551878,60.869565,51.918402,...,113986782,Wyoming,Teton,32886.299165,NaN,2.227379e+04,188154.050727,Wyoming (WY),6448803714,0.16%
3139,WY,Uinta,21118,2.018844e+09,26662000.0,2081.651840,4.458528,7.702101,30.434783,52.554202,...,113044654,Wyoming,Uinta,175826.688938,NaN,4.847070e+04,39079.410356,Wyoming (WY),6448803714,0.16%
3140,WY,Washakie,8533,8.867450e+08,43019000.0,2238.665800,5.182977,11.807766,47.826087,51.484398,...,45677149,Wyoming,Washakie,223781.785275,NaN,1.598940e+04,22528.317388,Wyoming (WY),6448803714,0.16%


In [121]:
df3 = df3.drop(columns='s')
df3

,state,county,population,buildvalue,agrivalue,area,risk_score,risk_npctl,risk_spctl,resl_score,...,revenue_per_person,state_funding,state.1,county.1,drgt_ealt,hrcn_ealt,swnd_ealt,ltng_ealt,funding,%_of_total
0,AL,Autauga,54571,5.075584e+09,21460000.0,594.448312,5.748777,16.359007,5.970149,55.529800,...,3756,204968676,Kentucky,Johnson,1825.417662,3272.974160,1.548040e+05,46448.730787,54697779694,1.33%
1,AL,Baldwin,182265,2.314295e+10,120383000.0,1589.815853,13.557905,79.344367,82.089552,54.759202,...,3756,684587340,Kentucky,Kenton,4641.483953,9185.266739,1.874503e+06,37762.813970,54697779694,1.33%
2,AL,Barbour,27457,2.206426e+09,105565000.0,885.001636,9.419639,52.959898,41.791045,50.781998,...,3756,103128492,Kentucky,Knott,1555.704677,615.508974,1.448994e+05,32220.028068,54697779694,1.33%
3,AL,Bibb,22915,1.643641e+09,4199000.0,622.460425,4.346314,7.192871,1.492537,51.807600,...,3756,86068740,Kentucky,Knox,17459.131710,2594.946558,3.297535e+05,105586.063748,54697779694,1.33%
4,AL,Blount,57322,4.794278e+09,242855000.0,644.821022,9.362945,52.546149,40.298507,53.357400,...,3756,215301432,Kentucky,Larue,23440.487786,17810.245704,3.498765e+05,38895.410938,54697779694,1.33%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,WY,Sweetwater,43806,4.583896e+09,16454000.0,10426.919825,2.572885,1.845958,13.043478,51.255002,...,5353,234493518,Wyoming,Sweetwater,67258.895675,NaN,2.931848e+04,36578.975784,6448803714,0.16%
3138,WY,Teton,21294,3.476594e+09,17582000.0,3996.855337,6.513988,23.551878,60.869565,51.918402,...,5353,113986782,Wyoming,Teton,32886.299165,NaN,2.227379e+04,188154.050727,6448803714,0.16%
3139,WY,Uinta,21118,2.018844e+09,26662000.0,2081.651840,4.458528,7.702101,30.434783,52.554202,...,5353,113044654,Wyoming,Uinta,175826.688938,NaN,4.847070e+04,39079.410356,6448803714,0.16%
3140,WY,Washakie,8533,8.867450e+08,43019000.0,2238.665800,5.182977,11.807766,47.826087,51.484398,...,5353,45677149,Wyoming,Washakie,223781.785275,NaN,1.598940e+04,22528.317388,6448803714,0.16%


In [122]:
df3[df3.state == 'SD'][['hurricane_score', 'hurricane_events', 'hurricane_freq', 'hrcn_ealt']].sum()

hurricane_score          0.000000
hurricane_events         0.000000
hurricane_freq           0.000000
hrcn_ealt           221453.953738
dtype: float64

In [123]:
df3 = df3.rename(columns={'drgt_ealt':'drought_loss', 'hrcn_ealt':'hurricane_loss'})
df3

,state,county,population,buildvalue,agrivalue,area,risk_score,risk_npctl,risk_spctl,resl_score,...,revenue_per_person,state_funding,state.1,county.1,drought_loss,hurricane_loss,swnd_ealt,ltng_ealt,funding,%_of_total
0,AL,Autauga,54571,5.075584e+09,21460000.0,594.448312,5.748777,16.359007,5.970149,55.529800,...,3756,204968676,Kentucky,Johnson,1825.417662,3272.974160,1.548040e+05,46448.730787,54697779694,1.33%
1,AL,Baldwin,182265,2.314295e+10,120383000.0,1589.815853,13.557905,79.344367,82.089552,54.759202,...,3756,684587340,Kentucky,Kenton,4641.483953,9185.266739,1.874503e+06,37762.813970,54697779694,1.33%
2,AL,Barbour,27457,2.206426e+09,105565000.0,885.001636,9.419639,52.959898,41.791045,50.781998,...,3756,103128492,Kentucky,Knott,1555.704677,615.508974,1.448994e+05,32220.028068,54697779694,1.33%
3,AL,Bibb,22915,1.643641e+09,4199000.0,622.460425,4.346314,7.192871,1.492537,51.807600,...,3756,86068740,Kentucky,Knox,17459.131710,2594.946558,3.297535e+05,105586.063748,54697779694,1.33%
4,AL,Blount,57322,4.794278e+09,242855000.0,644.821022,9.362945,52.546149,40.298507,53.357400,...,3756,215301432,Kentucky,Larue,23440.487786,17810.245704,3.498765e+05,38895.410938,54697779694,1.33%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,WY,Sweetwater,43806,4.583896e+09,16454000.0,10426.919825,2.572885,1.845958,13.043478,51.255002,...,5353,234493518,Wyoming,Sweetwater,67258.895675,NaN,2.931848e+04,36578.975784,6448803714,0.16%
3138,WY,Teton,21294,3.476594e+09,17582000.0,3996.855337,6.513988,23.551878,60.869565,51.918402,...,5353,113986782,Wyoming,Teton,32886.299165,NaN,2.227379e+04,188154.050727,6448803714,0.16%
3139,WY,Uinta,21118,2.018844e+09,26662000.0,2081.651840,4.458528,7.702101,30.434783,52.554202,...,5353,113044654,Wyoming,Uinta,175826.688938,NaN,4.847070e+04,39079.410356,6448803714,0.16%
3140,WY,Washakie,8533,8.867450e+08,43019000.0,2238.665800,5.182977,11.807766,47.826087,51.484398,...,5353,45677149,Wyoming,Washakie,223781.785275,NaN,1.598940e+04,22528.317388,6448803714,0.16%


In [124]:
df3.swnd_ealt+df3.ltng_ealt

0       2.012527e+05
1       1.912266e+06
2       1.771194e+05
3       4.353395e+05
4       3.887720e+05
            ...     
3137    6.589745e+04
3138    2.104278e+05
3139    8.755011e+04
3140    3.851772e+04
3141    8.002449e+04
Length: 3142, dtype: float64

In [125]:
df3['storm_loss'] = df3.swnd_ealt+df3.ltng_ealt

In [126]:
df3

,state,county,population,buildvalue,agrivalue,area,risk_score,risk_npctl,risk_spctl,resl_score,...,state_funding,state.1,county.1,drought_loss,hurricane_loss,swnd_ealt,ltng_ealt,funding,%_of_total,storm_loss
0,AL,Autauga,54571,5.075584e+09,21460000.0,594.448312,5.748777,16.359007,5.970149,55.529800,...,204968676,Kentucky,Johnson,1825.417662,3272.974160,1.548040e+05,46448.730787,54697779694,1.33%,2.012527e+05
1,AL,Baldwin,182265,2.314295e+10,120383000.0,1589.815853,13.557905,79.344367,82.089552,54.759202,...,684587340,Kentucky,Kenton,4641.483953,9185.266739,1.874503e+06,37762.813970,54697779694,1.33%,1.912266e+06
2,AL,Barbour,27457,2.206426e+09,105565000.0,885.001636,9.419639,52.959898,41.791045,50.781998,...,103128492,Kentucky,Knott,1555.704677,615.508974,1.448994e+05,32220.028068,54697779694,1.33%,1.771194e+05
3,AL,Bibb,22915,1.643641e+09,4199000.0,622.460425,4.346314,7.192871,1.492537,51.807600,...,86068740,Kentucky,Knox,17459.131710,2594.946558,3.297535e+05,105586.063748,54697779694,1.33%,4.353395e+05
4,AL,Blount,57322,4.794278e+09,242855000.0,644.821022,9.362945,52.546149,40.298507,53.357400,...,215301432,Kentucky,Larue,23440.487786,17810.245704,3.498765e+05,38895.410938,54697779694,1.33%,3.887720e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,WY,Sweetwater,43806,4.583896e+09,16454000.0,10426.919825,2.572885,1.845958,13.043478,51.255002,...,234493518,Wyoming,Sweetwater,67258.895675,NaN,2.931848e+04,36578.975784,6448803714,0.16%,6.589745e+04
3138,WY,Teton,21294,3.476594e+09,17582000.0,3996.855337,6.513988,23.551878,60.869565,51.918402,...,113986782,Wyoming,Teton,32886.299165,NaN,2.227379e+04,188154.050727,6448803714,0.16%,2.104278e+05
3139,WY,Uinta,21118,2.018844e+09,26662000.0,2081.651840,4.458528,7.702101,30.434783,52.554202,...,113044654,Wyoming,Uinta,175826.688938,NaN,4.847070e+04,39079.410356,6448803714,0.16%,8.755011e+04
3140,WY,Washakie,8533,8.867450e+08,43019000.0,2238.665800,5.182977,11.807766,47.826087,51.484398,...,45677149,Wyoming,Washakie,223781.785275,NaN,1.598940e+04,22528.317388,6448803714,0.16%,3.851772e+04


In [127]:
df3.columns

Index(['state', 'county', 'population', 'buildvalue', 'agrivalue', 'area',
       'risk_score', 'risk_npctl', 'risk_spctl', 'resl_score', 'resl_ratng',
       'resl_npctl', 'resl_spctl', 'resl_value', 'drought_events',
       'drought_freq', 'drought_score', 'hurricane_events', 'hurricane_freq',
       'hurricane_score', 'storm_events', 'storm_freq', 'storm_score',
       'full_state', 'revenue_per_person', 'state_funding', 'state.1',
       'county.1', 'drought_loss', 'hurricane_loss', 'swnd_ealt', 'ltng_ealt',
       'funding', '%_of_total', 'storm_loss'],
      dtype='object')

In [128]:
df3 = df3.drop(columns={'resl_spctl','resl_npctl','risk_npctl','risk_spctl','resl_ratng','swnd_ealt', 'ltng_ealt'})
df3

,state,county,population,buildvalue,agrivalue,area,risk_score,resl_score,resl_value,drought_events,...,full_state,revenue_per_person,state_funding,state.1,county.1,drought_loss,hurricane_loss,funding,%_of_total,storm_loss
0,AL,Autauga,54571,5.075584e+09,21460000.0,594.448312,5.748777,55.529800,2.77649,651.0,...,Alabama,3756,204968676,Kentucky,Johnson,1825.417662,3272.974160,54697779694,1.33%,2.012527e+05
1,AL,Baldwin,182265,2.314295e+10,120383000.0,1589.815853,13.557905,54.759202,2.73796,434.0,...,Alabama,3756,684587340,Kentucky,Kenton,4641.483953,9185.266739,54697779694,1.33%,1.912266e+06
2,AL,Barbour,27457,2.206426e+09,105565000.0,885.001636,9.419639,50.781998,2.53910,1043.0,...,Alabama,3756,103128492,Kentucky,Knott,1555.704677,615.508974,54697779694,1.33%,1.771194e+05
3,AL,Bibb,22915,1.643641e+09,4199000.0,622.460425,4.346314,51.807600,2.59038,714.0,...,Alabama,3756,86068740,Kentucky,Knox,17459.131710,2594.946558,54697779694,1.33%,4.353395e+05
4,AL,Blount,57322,4.794278e+09,242855000.0,644.821022,9.362945,53.357400,2.66787,665.0,...,Alabama,3756,215301432,Kentucky,Larue,23440.487786,17810.245704,54697779694,1.33%,3.887720e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,WY,Sweetwater,43806,4.583896e+09,16454000.0,10426.919825,2.572885,51.255002,2.56275,2156.0,...,Wyoming,5353,234493518,Wyoming,Sweetwater,67258.895675,NaN,6448803714,0.16%,6.589745e+04
3138,WY,Teton,21294,3.476594e+09,17582000.0,3996.855337,6.513988,51.918402,2.59592,1029.0,...,Wyoming,5353,113986782,Wyoming,Teton,32886.299165,NaN,6448803714,0.16%,2.104278e+05
3139,WY,Uinta,21118,2.018844e+09,26662000.0,2081.651840,4.458528,52.554202,2.62771,1386.0,...,Wyoming,5353,113044654,Wyoming,Uinta,175826.688938,NaN,6448803714,0.16%,8.755011e+04
3140,WY,Washakie,8533,8.867450e+08,43019000.0,2238.665800,5.182977,51.484398,2.57422,1309.0,...,Wyoming,5353,45677149,Wyoming,Washakie,223781.785275,NaN,6448803714,0.16%,3.851772e+04


In [101]:
df3.columns

Index(['state', 'county', 'population', 'buildvalue', 'agrivalue', 'area',
       'risk_score', 'resl_score', 'resl_value', 'drought_events',
       'drought_freq', 'drought_score', 'hurricane_events', 'hurricane_freq',
       'hurricane_score', 'storm_events', 'storm_freq', 'storm_score',
       'full_state', 'revenue_per_person', 'state_funding', 'drought_loss',
       'hurricane_loss', 'funding', '%_of_total', 'storm_loss'],
      dtype='object')

In [ ]:
df3.drop(columns={''})

In [131]:
df3 = df3[['full_state','state', 'county', 'population', 'revenue_per_person', 'state_funding','funding','buildvalue', 'agrivalue', 'area',
     'risk_score', 'resl_score', 'resl_value', 'drought_events',
       'drought_freq', 'drought_score', 'drought_loss','hurricane_events', 'hurricane_freq',
       'hurricane_score', 'hurricane_loss','storm_events', 'storm_freq', 'storm_score', 'storm_loss']]

In [132]:
df3

,full_state,state,county,population,revenue_per_person,state_funding,funding,buildvalue,agrivalue,area,...,drought_score,drought_loss,hurricane_events,hurricane_freq,hurricane_score,hurricane_loss,storm_events,storm_freq,storm_score,storm_loss
0,Alabama,AL,Autauga,54571,3756,204968676,54697779694,5.075584e+09,21460000.0,594.448312,...,4.191694,1825.417662,8.0,0.056426,2.946097,3272.974160,1775.0,65.740741,7.631697,2.012527e+05
1,Alabama,AL,Baldwin,182265,3756,684587340,54697779694,2.314295e+10,120383000.0,1589.815853,...,12.636110,4641.483953,44.0,0.208845,12.704112,9185.266739,2934.0,108.666667,23.224606,1.912266e+06
2,Alabama,AL,Barbour,27457,3756,103128492,54697779694,2.206426e+09,105565000.0,885.001636,...,18.092328,1555.704677,13.0,0.073180,7.311136,615.508974,1687.0,62.481481,12.471689,1.771194e+05
3,Alabama,AL,Bibb,22915,3756,86068740,54697779694,1.643641e+09,4199000.0,622.460425,...,2.374610,17459.131710,7.0,0.043792,1.884830,2594.946558,1911.0,70.777778,5.382791,4.353395e+05
4,Alabama,AL,Blount,57322,3756,215301432,54697779694,4.794278e+09,242855000.0,644.821022,...,11.941503,23440.487786,2.0,0.012192,5.013860,17810.245704,2081.0,77.074074,13.540158,3.887720e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,Wyoming,WY,Sweetwater,43806,5353,234493518,6448803714,4.583896e+09,16454000.0,10426.919825,...,2.840916,67258.895675,0.0,0.000000,0.000000,NaN,296.0,10.962963,3.481503,6.589745e+04
3138,Wyoming,WY,Teton,21294,5353,113986782,6448803714,3.476594e+09,17582000.0,3996.855337,...,2.707191,32886.299165,0.0,0.000000,0.000000,NaN,256.0,9.481481,6.210718,2.104278e+05
3139,Wyoming,WY,Uinta,21118,5353,113044654,6448803714,2.018844e+09,26662000.0,2081.651840,...,5.762984,175826.688938,0.0,0.000000,0.000000,NaN,350.0,12.962963,5.590674,8.755011e+04
3140,Wyoming,WY,Washakie,8533,5353,45677149,6448803714,8.867450e+08,43019000.0,2238.665800,...,10.952769,223781.785275,0.0,0.000000,0.000000,NaN,249.0,9.222222,7.378384,3.851772e+04


In [133]:
df3.columns

Index(['full_state', 'state', 'county', 'population', 'revenue_per_person',
       'state_funding', 'funding', 'buildvalue', 'agrivalue', 'area',
       'risk_score', 'resl_score', 'resl_value', 'drought_events',
       'drought_freq', 'drought_score', 'drought_loss', 'hurricane_events',
       'hurricane_freq', 'hurricane_score', 'hurricane_loss', 'storm_events',
       'storm_freq', 'storm_score', 'storm_loss'],
      dtype='object')

In [134]:
df3.to_csv('finalcsv.csv')

---

In [54]:
ww = pd.read_csv('NRI_Table_Counties.csv')

In [56]:
ww

,OID_,NRI_ID,STATE,STATEABBRV,STATEFIPS,COUNTY,COUNTYTYPE,COUNTYFIPS,STCOFIPS,POPULATION,...,WNTW_EALB,WNTW_EALP,WNTW_EALPE,WNTW_EALA,WNTW_EALT,WNTW_EALS,WNTW_EALR,WNTW_RISKS,WNTW_RISKR,NRI_VER
0,1,C21115,Kentucky,KY,21,Johnson,County,115,21115,23356,...,781.792240,0.006129,46577.171552,4.235939,47363.199731,19.448529,Relatively Moderate,14.131237,Relatively Low,November 2021
1,2,C21117,Kentucky,KY,21,Kenton,County,117,21117,159720,...,18249.808380,0.006048,45965.118059,44.606252,64259.532691,21.530408,Relatively Moderate,12.470040,Relatively Low,November 2021
2,3,C21119,Kentucky,KY,21,Knott,County,119,21119,16346,...,249.454293,0.004021,30560.277236,0.023091,30809.754620,16.851393,Relatively Low,14.466270,Relatively Low,November 2021
3,4,C21121,Kentucky,KY,21,Knox,County,121,21121,31883,...,17055.601129,0.005838,44371.625150,0.082573,61427.308851,21.209328,Relatively Moderate,19.585915,Relatively Moderate,November 2021
4,5,C21123,Kentucky,KY,21,Larue,County,123,21123,14193,...,9662.424572,0.000390,2961.292206,246.668438,12870.385216,12.597091,Relatively Low,7.715952,Very Low,November 2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,3138,C56037,Wyoming,WY,56,Sweetwater,County,37,56037,43806,...,165.637265,0.003126,23755.525159,0.000041,23921.162465,15.488184,Relatively Low,5.189094,Very Low,November 2021
3138,3139,C56039,Wyoming,WY,56,Teton,County,39,56039,21294,...,2688.566845,0.001068,8118.192272,0.000078,10806.759196,11.884244,Relatively Low,4.816165,Very Low,November 2021
3139,3140,C56041,Wyoming,WY,56,Uinta,County,41,56041,21118,...,63.272547,0.010158,77199.713455,27.256486,77290.242488,22.897143,Relatively Moderate,11.296648,Relatively Low,November 2021
3140,3141,C56043,Wyoming,WY,56,Washakie,County,43,56043,8533,...,119.536607,0.000703,5345.785169,1.415514,5466.737290,9.469255,Very Low,8.193083,Very Low,November 2021


In [85]:
ww[ww.STATEABBRV == 'SD'][['HRCN_EALT']]

,HRCN_EALT
2320,NaN
2321,NaN
2322,NaN
2323,NaN
2324,NaN
...,...
2423,NaN
2424,NaN
2425,NaN
2426,NaN


In [87]:
np.where(ww[ww.STATEABBRV == 'SD'].HRCN_EALT>0)

(array([], dtype=int64),)

In [75]:
dd = ww[ww.HRCN_EALT > 0][['STATEABBRV','HRCN_EALT']]

In [78]:
dd.STATEABBRV.value_counts()

TX      233
GA      159
VA      133
KY      120
IL      102
NC      100
MO       99
TN       95
IN       92
OH       88
MS       82
AR       75
OK       75
MI       70
IA       70
AL       67
FL       67
PA       67
LA       64
NY       62
WV       55
SC       46
WI       34
MD       24
NJ       21
KS       19
ME       16
MA       14
VT       14
AZ       11
NM       11
NH       10
CT        8
HI        5
RI        5
DE        3
CA        3
D.C.      1
MN        1
Name: STATEABBRV, dtype: int64

In [79]:
dd[dd.STATEABBRV == 'IA']

,STATEABBRV,HRCN_EALT
761,IA,67250.476570
762,IA,38723.974598
763,IA,72229.303691
764,IA,16764.000081
765,IA,98518.354062
...,...,...
939,IA,151424.134502
940,IA,33016.028239
941,IA,116714.946902
943,IA,121238.245478


---

In [ ]:
pd.read_csv('')